### Overview

This JupyTer notebook demonstrates the extraction of metadata from all files of a given directory into a `.csv` file. It then performs analytics on the types of files contained in the directory

In [2]:
from file_processing import Directory

#### Defining the directory to analyze

Point to the directory of interest. For absolute paths, format using one of these formats: `r'C:\...\...'` or `'C:\\...\\'` or `'C:/.../...'`

In [3]:
dire = Directory('./tests/resources/similarity_test_files/')

#### Exploring the CSV report

`dire.generate_report()` accepts the following:

* `report_file` (str): Path to the output csv file
* `include_text` (bool; False): Whether to include text data from the files in the report
* `filters` (dict; None): Inclusion/exclusion criteria
* `keywords` (list; None): A list of keywords to count in the 'text' field of the metadata
* `migrate_filters` (dict; None): Adds a boolean column with 1 if the file satisfies the input criteria, else 0
* `open_files` (bool; True): Whether to open the files to extract file-specific metadata
* `split_metadata` (bool; False): Whether to split the file-specific metadata object such that each field has its own column
* `char_limit` (int; 3000): The cut-off length for each metadata field

Sample filter parameter:

```py
filter = {
    'exclude_str': ['.venv', '.pytest_cache', '.vscode', '__pycache__']         # direectories or specific files to EXCLUDE
    'include_str': ['file_processing', 'tests', '__init.py__'],                 # direectories or specific files to INCLUDE
    'extensions': ['.csv', '.docx', '.pptx', '.xlsx', '.pdf'],                  # file extensions to INCLUDE
    'exclude_extensions': ['.tmp', '', '.py'],                                  # file extensions to EXCLUDE
    'min_size': 10000,                                                          # minimum file size (in bytes)
    'max_size': 50000                                                           # maximum file size (in bytes)
}
```

In [ ]:
dire.generate_report(report_file='./tests/resources/sample_reports/report.csv', 
                    split_metadata=True, 
                    filters={'exclude_str': ['.venv', '.pytest_cache', '.vscode', '__pycache__', '.git']},
                    migrate_filters={'extensions': ['.csv', '.docx', '.pdf', '.pptx']},
                    include_text=True,
                    keywords=['Health', 'Canada', 'Classified', 'Protected'])

#### Filtering out the data of interest

In [ ]:
import pandas as pd

analytics = dire.generate_analytics(report_file='./tests/resources/sample_reports/analytics.csv', 
                                   filters={'exclude_str': ['.venv', '.pytest_cache', '.vscode', '__pycache__', '.git']})
df = pd.DataFrame.from_dict((analytics))
df

#### Visualizing the filtered data

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.yaxis.get_major_locator().set_params(integer=True)

width = 0.4

(df['size (MB)']).plot(kind='bar', color='#0072b2', ax=ax, width=width, position=1, label='Size (mb)')
df['count'].plot(kind='bar', color='#88ccee', ax=ax2, width=width, position=0, label='Count')

ax.set_ylabel('Size (mb)')
ax2.set_ylabel('Count')
ax.set_xlabel('File type')

plt.legend()
plt.title('File types by Count and Size (mb)')


In [ ]:
import numpy as np

labels = df.index
sizes = df['size (MB)']

cmap = plt.get_cmap("Pastel2")

plt.title('Pie chart of file types by size')
plt.pie(sizes, labels=labels, colors=cmap(np.arange(3)*4))
plt.show()

#### Identifying similar files

The below example uses **cosine** similarity (`threshold=0`) and returns a grid comparing every file against each other

In [23]:
dire = Directory('./tests/resources/similarity_test_files/')
dire.identify_duplicates(report_file='./tests/resources/sample_reports/similarity_cosine.csv',
                        filters={}, 
                        threshold=0,
                        top_n=0,
                        use_abs_path=True)

Processing files: 100%|██████████| 20/20 [00:00<00:00, 69.33file/s]


Alternatively, it is possible to use **FAISS** indexes by specifying a threshold

In [47]:
dire.identify_duplicates(report_file='./tests/resources/sample_reports/similarity_faiss.csv',
                        filters={'min_size': 1000}, 
                        threshold=0.5,
                        top_n=100,
                        use_abs_path=True)

Processing files: 100%|██████████| 20/20 [00:00<00:00, 62.51file/s]
c:\Users\BLUO\Downloads\report\.venv\lib\site-packages\pandas\core\internals\blocks.py:1616: RuntimeWarning: overflow encountered in multiply
  values = self.values.round(decimals)  # type: ignore[union-attr]


In [48]:
import pandas as pd

df = pd.read_csv('./tests/resources/sample_reports/similarity_faiss.csv', index_col=0)
# df.map(lambda x: pd.Series(x).between(-1, 1).all()).all().all()
df

,absolute_path,1_file,1,2_file,2,3_file,3,4_file,4,5_file,...,16_file,16,17_file,17,18_file,18,19_file,19,20_file,20
0,C:\Users\BLUO\Downloads\report\tests\resources...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\Users\BLUO\Downloads\report\tests\resources...,C:\Users\BLUO\Downloads\report\tests\resources...,0.57,C:\Users\BLUO\Downloads\report\tests\resources...,0.50,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\Users\BLUO\Downloads\report\tests\resources...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\Users\BLUO\Downloads\report\tests\resources...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\Users\BLUO\Downloads\report\tests\resources...,C:\Users\BLUO\Downloads\report\tests\resources...,0.73,C:\Users\BLUO\Downloads\report\tests\resources...,0.51,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,C:\Users\BLUO\Downloads\report\tests\resources...,C:\Users\BLUO\Downloads\report\tests\resources...,0.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,C:\Users\BLUO\Downloads\report\tests\resources...,C:\Users\BLUO\Downloads\report\tests\resources...,0.58,C:\Users\BLUO\Downloads\report\tests\resources...,0.55,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,C:\Users\BLUO\Downloads\report\tests\resources...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,C:\Users\BLUO\Downloads\report\tests\resources...,C:\Users\BLUO\Downloads\report\tests\resources...,0.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,C:\Users\BLUO\Downloads\report\tests\resources...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df[['1', '2', '3']].map(lambda x: pd.Series(x).dropna().between(-1, 1, inclusive='both').all()).all().all()

True